In [ ]:
%load_ext autoreload

import numpy as np
import time
import random
from matplotlib import pyplot as plt
%matplotlib inline
from PIL import Image
from datetime import datetime
from tqdm import tqdm
from pprint import pprint
from threading import Thread
from multiprocessing import Process
from object_detection.utils import label_map_util
from object_detection.core import standard_fields
from object_detection.utils import object_detection_evaluation

from CNNRobotLocalisation.SecondStage.second_stage_utils import *
from CNNRobotLocalisation.Utils.file_utils import *
from inference_utils import *
from TwoStageFramework import TwoStageFramework

NETWORK_FILES = {
    'FIRST_STAGE': '/home/lhoyer/cnn_robot_localization/training/first_stage_2/ssd_mobilenet2/exp1/background_diversity_coco/deploy_00/frozen_inference_graph.pb',
    #'COPTER_ID_NET': '/home/lhoyer/cnn_robot_localization/training_result_models/second_stage_2/sstage_default_copter_cat/2018-06-12-01-27/model-final.h5',
    #'SPHERO_ID_NET': '/home/lhoyer/cnn_robot_localization/training_result_models/second_stage_2/sstage_default_sphero_cat/2018-06-12-04-54/model-final.h5',
    #'YOUBOT_ID_NET': '/home/lhoyer/cnn_robot_localization/training_result_models/second_stage_2/sstage_default_youbot_cat/2018-06-12-14-50/model-final.h5',
    'COPTER_ID_NET': '/home/lhoyer/cnn_robot_localization/training_result_models/second_stage_2/sstage_amount_20000_copter_cat/2018-06-21-17-19/model-final.h5',
    'SPHERO_ID_NET': '/home/lhoyer/cnn_robot_localization/training_result_models/second_stage_2/sstage_amount_20000_sphero_cat/2018-06-22-13-27/model-final.h5',
    'YOUBOT_ID_NET': '/home/lhoyer/cnn_robot_localization/training_result_models/second_stage_2/sstage_amount_20000_youbot_cat/2018-06-20-14-12/model-final.h5',
    #'COPTER_ROT_NET': '/home/lhoyer/cnn_robot_localization/training_result_models/second_stage_2/sstage_default_copter_bin/2018-06-12-03-45/model-05-0.02.h5',
    #'SPHERO_ROT_NET': '/home/lhoyer/cnn_robot_localization/training_result_models/second_stage_2/sstage_default_sphero_bin/2018-06-12-05-56/model-05-0.00.h5',
    #'YOUBOT_ROT_NET': '/home/lhoyer/cnn_robot_localization/training_result_models/second_stage_2/sstage_default_youbot_bin/2018-06-12-07-14/model-05-0.02.h5',
    'COPTER_ROT_NET': '/home/lhoyer/cnn_robot_localization/training_result_models/second_stage_2/sstage_amount_20000_copter_bin/2018-06-21-09-16/model-05-0.03.h5',
    'SPHERO_ROT_NET': '/home/lhoyer/cnn_robot_localization/training_result_models/second_stage_2/sstage_amount_20000_sphero_bin/2018-06-21-01-31/model-05-0.00.h5',
    'YOUBOT_ROT_NET': '/home/lhoyer/cnn_robot_localization/training_result_models/second_stage_2/sstage_amount_20000_youbot_bin/2018-06-21-03-55/model-05-0.00.h5',
}
PATH_TO_LABELS = "/home/lhoyer/cnn_robot_localization/CNNRobotLocalisation/LabelMaps/robot_label_map.pbtxt"
TYPE = 'copter'
PATH_TO_TEST_IMAGES_DIR = '/home/lhoyer/cnn_robot_localization/data/Validation/180420/ValidationWithoutDistr/180420_{}'.format(TYPE)
TEST_IMAGE_PATHS = get_recursive_file_list(PATH_TO_TEST_IMAGES_DIR, file_extensions=[".jpg"])

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
num_classes = label_map_util.get_max_label_map_index(
                        label_map_util.load_labelmap(PATH_TO_LABELS)) + 1
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(PATH_TO_LABELS)


In [ ]:
two_stage_inference = TwoStageFramework(NETWORK_FILES)

In [ ]:
def convert_pascal_voc_to_groundtruth_dict(voc):
    #print(voc['object'])
    boxes, classes, poses = [], [], []
    for o in voc['object']:
        subclass_text = o['name'].replace('/','_')
        if 'undefined' in o['name'] or ('pose_defined' in o and o['pose_defined']==str(0)):
            print(voc)
            return False
        classes.append(label_map_dict[subclass_text])
        boxes.append([int(o['bndbox']['ymin']) / 1200, 
                      int(o['bndbox']['xmin']) / 1600, 
                      int(o['bndbox']['ymax']) / 1200,
                      int(o['bndbox']['xmax']) / 1600])
        poses.append(o['pose'])
    groundtruth_dict = {}
    groundtruth_dict['groundtruth_classes'] = np.asarray(classes, dtype=np.int32)
    groundtruth_dict['groundtruth_boxes'] = np.asarray(boxes, dtype=np.float32)
    groundtruth_dict['poses'] = np.asarray(poses)
    return groundtruth_dict
    
convert_pascal_voc_to_groundtruth_dict({'segmented': '0', 'source': {'database': 'Unknown'}, 'object': [{'name': 'copter/green_white', 'truncated': '0', 'bndbox': {'ymax': '936', 'ymin': '846', 'xmin': '1083', 'xmax': '1177'}, 'difficult': '0', 'pose': '2'}], 'path': '/media/data/LocalizationData/Output/EvalData1/180420_by_nl_eval_copter_led.avi/BoundingBoxes/frame000000.jpg', 'filename': 'frame000000.jpg', 'size': {'width': '1600', 'height': '1200', 'depth': '3'}, 'folder': 'BoundingBoxes'})

In [ ]:
from object_detection.utils import np_box_ops

def matching_pred(pred, gt):
    if len(pred['detection_boxes']) > 1:
        print('More than one detection: {}'.format(len(pred['detection_boxes'])))
    iou = np_box_ops.iou(pred['detection_boxes'], gt['groundtruth_boxes'])
    #print('IOU: {}'.format(iou))
    try:
        matching_idx = np.array(np.where(iou > 0.5)).T[0][0]
    except IndexError:
        return None
    #print(matching_idx)
    #print('Pred scores: {}'.format(pred['detection_scores']))
    return pred['detection_classes'][matching_idx], pred['poses'][matching_idx], matching_idx

In [ ]:
%autoreload

i = 0
evaluator = object_detection_evaluation.PascalDetectionEvaluator(categories)
pred_angles, gt_angles = [], []
id_row_distr, rot_row_distr = PredictionRowDistribution(), PredictionRowDistribution()

random.seed(0)
#shuffled_img_paths = random.sample(TEST_IMAGE_PATHS, len(TEST_IMAGE_PATHS))
for image_path in tqdm(TEST_IMAGE_PATHS[:]):
    print(image_path)
    xml_path = image_path.replace('.jpg','.xml')
    voc = parseXML(xml_path)['annotation']
    image_hr = Image.open(image_path).convert('RGB')
    threshold = 0.1
    if TYPE == 'copter': threshold = 0.2
    if TYPE == 'sphero': threshold = 0.3
    if TYPE == 'youbot': threshold = 0.1
    output_dict = two_stage_inference.infer(image_hr, threshold)
    
    # Evaluate subclass detection mAP
    output_dict['detection_classes'] = np.asarray(output_dict['sub_classes'])
    output_dict['detection_scores'] = np.asarray(output_dict['sub_scores'])
    
    if len(output_dict['detection_boxes']) == 0:
        output_dict['detection_boxes'] = np.zeros((0, 4))
    output_dict['detection_boxes'] = np.atleast_2d(output_dict['detection_boxes'])
    gt_dict = convert_pascal_voc_to_groundtruth_dict(voc)
    #if len(output_dict['detection_classes']) > 1:
    #    print(output_dict)
    #    print(gt_dict)
    if gt_dict == False:
        print('Skip {}'.format(image_path))
        continue
    evaluator.add_single_ground_truth_image_info(image_path,gt_dict)
    evaluator.add_single_detected_image_info(image_path, output_dict)
    
    assert len(gt_dict['groundtruth_classes']) == 1
    #if len(output_dict['detection_classes']) > 0:
    matching_prediction = matching_pred(output_dict, gt_dict)
    
    # Evaluate orientation error    
    if matching_prediction != None:
        #print(matching_prediction)
        pred_angles.append(matching_prediction[1])
        gt_angles.append(int(gt_dict['poses'][0]))
        
    # Evaluate error distribution
    correct_id, correct_rot = False, False
    if matching_prediction != None:
        if matching_prediction[0] == gt_dict['groundtruth_classes'][0]:
            correct_id = True  
        else:
            print(matching_prediction[0])
            print(gt_dict)
        if abs(np_angle_diff2(matching_prediction[1], int(gt_dict['poses'][0]))) <= 5:
            correct_rot = True
        #else:
        #    print(matching_prediction[1])
        #    print(int(gt_dict['poses'][0]))
    id_row_distr.add_prediction(correct_id)
    rot_row_distr.add_prediction(correct_rot)


In [ ]:
%autoreload
id_row_distr.finalize()
rot_row_distr.finalize()
#print('Object not recognized: {}'.format(obj_not_rec))
print({
    'id_pos': id_row_distr.pos_in_row,
    'id_neg': id_row_distr.neg_in_row,
    'rot_pos': rot_row_distr.pos_in_row,
    'rot_neg': rot_row_distr.pos_in_row,
})
pprint(evaluator.evaluate())
print('Orientation MAE:')
print(np.mean(np.abs(np_angle_diff2(np.array(gt_angles), np.array(pred_angles)))))

In [ ]:
# Find best threshold on subset
%autoreload

min_true_score = 1.0
matching_scores = []

random.seed(0)
shuffled_img_paths = random.sample(TEST_IMAGE_PATHS, len(TEST_IMAGE_PATHS))
for image_path in tqdm(shuffled_img_paths[0:200]):
    xml_path = image_path.replace('.jpg','.xml')
    voc = parseXML(xml_path)['annotation']
    image_hr = Image.open(image_path).convert('RGB')
    threshold = 0.1
    output_dict = two_stage_inference.infer(image_hr, threshold)
    output_dict['detection_classes'] = np.asarray(output_dict['sub_classes'])
    output_dict['detection_scores'] = np.asarray(output_dict['sub_scores'])
    
    if len(output_dict['detection_boxes']) == 0:
        output_dict['detection_boxes'] = np.zeros((0, 4))
    output_dict['detection_boxes'] = np.atleast_2d(output_dict['detection_boxes'])
    gt_dict = convert_pascal_voc_to_groundtruth_dict(voc)
    if gt_dict == False:
        print('Skip {}'.format(image_path))
        continue
    
    assert len(gt_dict['groundtruth_classes']) == 1
    matching_prediction = matching_pred(output_dict, gt_dict)
    if matching_prediction != None:
        score = output_dict['super_scores'][matching_prediction[2]]
        if score < min_true_score:
            min_true_score = score
        print('Matching: {}'.format(score))
        matching_scores.append(score)

plt.hist(matching_scores, 20)
plt.show()
print('Min true score: {}'.format(min_true_score))

In [ ]:
# Results for further processing

#copter_distr = {'rot_neg': [95, 28, 19, 9, 64, 230, 37, 33, 50, 182, 9, 171, 24, 161, 43, 29, 13, 5, 1, 138, 91, 20, 275, 2, 38, 55, 44, 156, 173, 31, 21, 1, 94, 6, 39, 70, 14, 107, 36, 94, 289, 224, 35, 13, 8, 16, 34, 4, 1, 132, 96], 'id_neg': [1, 1, 2, 1, 1, 3, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'rot_pos': [95, 28, 19, 9, 64, 230, 37, 33, 50, 182, 9, 171, 24, 161, 43, 29, 13, 5, 1, 138, 91, 20, 275, 2, 38, 55, 44, 156, 173, 31, 21, 1, 94, 6, 39, 70, 14, 107, 36, 94, 289, 224, 35, 13, 8, 16, 34, 4, 1, 132, 96], 'id_pos': [154, 64, 5, 181, 124, 1, 227, 3, 244, 4, 188, 11, 9, 67, 75, 26, 156, 1, 114, 156, 15, 24, 25, 8, 48, 44, 1, 223, 31, 3, 6, 1, 106, 1, 2, 291, 12, 9, 23, 23, 116, 490, 12, 31, 37, 5, 4, 63, 4, 3, 8, 78]}
#sphero_distr = {'id_neg': [1, 2, 1, 4, 5, 1, 1, 5, 3, 1, 6, 1, 1, 1, 2, 5, 2, 1, 1, 1, 3, 5, 1, 3, 2, 3, 1, 1, 3, 2, 3, 7, 1, 1, 1, 2, 6, 3, 1, 1, 2, 2, 1, 1, 1, 1, 3, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 4, 1, 6, 5, 3, 1, 1, 1, 1, 1, 8, 1, 2, 1, 1, 1, 1, 1, 4, 11, 5, 1, 1, 1, 1, 1, 1, 1, 1, 2, 12, 1, 2, 1, 1, 3, 1, 15, 2, 1, 1, 1, 1, 1, 3, 3, 3, 2, 1, 1, 1, 1, 5, 9, 1, 2, 1, 1, 3, 1, 10, 4, 5, 1, 5, 1, 1, 1, 2, 1, 1, 1, 15, 1, 1, 3, 6, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 7, 3, 2, 2, 1, 7, 4, 3, 3, 5, 2, 1, 3, 1, 13, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 7, 2, 1, 1, 1, 1, 2, 6, 6, 1, 1, 13, 2, 14, 2, 2, 1, 1, 3, 1, 1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 2, 1, 5, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 3, 1, 15, 1, 2, 1, 2, 2, 4, 1, 1, 2, 2, 2, 4, 7, 1, 2, 1, 1, 3, 1, 2, 1, 1, 1, 1, 1, 4, 1, 2, 1, 1, 1, 4, 1, 2, 1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 5, 1, 3, 6, 4, 1, 3, 1, 1, 1, 1, 2, 2], 'rot_neg': [1, 2, 2, 5, 1, 1, 2, 4, 6, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 2, 2, 1, 3, 1, 1, 3, 1, 2, 3, 1, 1, 1, 2, 3, 2, 3, 2, 3, 1, 2, 1, 2, 2, 1, 4, 3, 4, 3, 1, 12, 1, 2, 3, 1, 1, 1, 7, 2, 1, 1, 2, 1, 1, 5, 1, 1, 2, 1, 1, 1, 1, 1, 4, 4, 3, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 3, 1, 4, 2, 1, 3, 1, 3, 3, 1, 4, 7, 1, 2, 1, 1, 1, 3, 1, 2, 2, 2, 3, 7, 1, 8, 1, 1, 1, 3, 1, 1, 1, 2, 1, 5, 5, 1, 1, 11, 2, 2, 7, 1, 1, 1, 2, 2, 1, 3, 2, 1, 2, 1, 2, 1, 1, 1, 3, 3, 2, 2, 2, 1, 1, 2, 2, 1, 3, 1, 2, 2, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 4, 1, 2, 2, 8, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 3, 3, 2, 1, 2, 1, 2, 1, 1, 1, 2, 3, 1, 1, 8, 1, 1, 1, 3, 1, 3, 1, 1, 1, 3, 1, 5, 3, 3, 1, 11, 1, 2, 1, 1, 4, 1, 1, 1, 1, 2, 1, 2, 2, 3, 3, 8, 1, 6, 2, 3, 1, 1, 2, 5, 17, 17, 1, 3, 3, 6, 2, 2, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, 3, 8, 1, 2, 3, 1, 3, 1, 1, 5, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 3, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 3, 1, 3, 5, 3, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 2, 3, 2, 2, 3, 1, 1, 1, 10, 1, 1, 1, 5, 1, 1, 2, 1, 5, 1, 1, 2, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 7, 1, 1, 1, 2, 4, 1, 2, 2, 4, 1, 1, 1, 3, 3, 3, 1, 1, 1, 3, 1, 4, 1, 1, 1, 1, 2, 2, 2, 1, 1, 5, 5, 5, 2, 4, 5, 1, 1, 1, 3, 1, 4, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 3, 2, 4, 1, 1, 4, 1, 1], 'id_pos': [48, 12, 1, 3, 3, 4, 2, 5, 7, 1, 10, 2, 10, 10, 2, 1, 2, 13, 12, 25, 5, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 8, 6, 10, 15, 2, 1, 1, 1, 7, 3, 5, 19, 37, 1, 2, 1, 2, 2, 1, 5, 1, 4, 2, 7, 4, 24, 5, 5, 1, 1, 3, 14, 2, 5, 2, 12, 1, 4, 1, 1, 9, 2, 3, 1, 1, 2, 1, 3, 5, 1, 9, 4, 3, 10, 13, 1, 4, 2, 1, 1, 1, 1, 6, 1, 1, 1, 1, 4, 2, 74, 1, 5, 3, 1, 7, 1, 3, 2, 4, 2, 3, 8, 4, 1, 7, 3, 2, 2, 1, 6, 2, 1, 3, 9, 9, 12, 5, 2, 2, 4, 3, 3, 2, 4, 2, 13, 6, 1, 3, 1, 4, 2, 8, 2, 1, 3, 4, 1, 2, 1, 2, 1, 16, 5, 2, 1, 1, 2, 13, 1, 24, 1, 1, 2, 3, 30, 3, 1, 2, 12, 2, 2, 4, 20, 12, 2, 14, 4, 1, 1, 1, 4, 8, 1, 22, 3, 44, 1, 23, 2, 1, 1, 1, 50, 2, 11, 1, 1, 10, 1, 2, 1, 51, 12, 10, 1, 1, 5, 1, 2, 3, 9, 6, 1, 1, 1, 4, 1, 9, 7, 4, 2, 2, 2, 50, 2, 2, 5, 7, 7, 4, 1, 2, 2, 1, 6, 20, 7, 3, 47, 1, 7, 2, 4, 3, 1, 3, 7, 1, 18, 1, 1, 9, 11, 4, 36, 1, 6, 12, 9, 18, 3, 20, 4, 14, 3, 1, 13, 7, 7, 1, 1, 17, 10, 23, 2, 7, 4, 4, 2, 7, 9, 11, 22, 4, 4, 4, 1, 12, 1, 2, 2, 1, 1, 1, 5, 4, 22, 8, 32, 1, 4, 65], 'rot_pos': [1, 2, 2, 5, 1, 1, 2, 4, 6, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 2, 2, 1, 3, 1, 1, 3, 1, 2, 3, 1, 1, 1, 2, 3, 2, 3, 2, 3, 1, 2, 1, 2, 2, 1, 4, 3, 4, 3, 1, 12, 1, 2, 3, 1, 1, 1, 7, 2, 1, 1, 2, 1, 1, 5, 1, 1, 2, 1, 1, 1, 1, 1, 4, 4, 3, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 3, 1, 4, 2, 1, 3, 1, 3, 3, 1, 4, 7, 1, 2, 1, 1, 1, 3, 1, 2, 2, 2, 3, 7, 1, 8, 1, 1, 1, 3, 1, 1, 1, 2, 1, 5, 5, 1, 1, 11, 2, 2, 7, 1, 1, 1, 2, 2, 1, 3, 2, 1, 2, 1, 2, 1, 1, 1, 3, 3, 2, 2, 2, 1, 1, 2, 2, 1, 3, 1, 2, 2, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 4, 1, 2, 2, 8, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 3, 3, 2, 1, 2, 1, 2, 1, 1, 1, 2, 3, 1, 1, 8, 1, 1, 1, 3, 1, 3, 1, 1, 1, 3, 1, 5, 3, 3, 1, 11, 1, 2, 1, 1, 4, 1, 1, 1, 1, 2, 1, 2, 2, 3, 3, 8, 1, 6, 2, 3, 1, 1, 2, 5, 17, 17, 1, 3, 3, 6, 2, 2, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, 3, 8, 1, 2, 3, 1, 3, 1, 1, 5, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 3, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 3, 1, 3, 1, 3, 5, 3, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 2, 3, 2, 2, 3, 1, 1, 1, 10, 1, 1, 1, 5, 1, 1, 2, 1, 5, 1, 1, 2, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 7, 1, 1, 1, 2, 4, 1, 2, 2, 4, 1, 1, 1, 3, 3, 3, 1, 1, 1, 3, 1, 4, 1, 1, 1, 1, 2, 2, 2, 1, 1, 5, 5, 5, 2, 4, 5, 1, 1, 1, 3, 1, 4, 2, 1, 2, 2, 2, 1, 1, 1, 1, 1, 3, 2, 4, 1, 1, 4, 1, 1]}
#youbot_distr = {'rot_neg': [2, 1, 9, 1, 1, 2, 1, 4, 2, 1, 4, 15, 2, 7, 2, 5, 1, 3, 1, 19, 3, 46, 2, 1, 10, 7, 2, 4, 23, 1, 10, 15, 55, 1, 1, 1, 2, 7, 14, 7, 1, 7, 6, 30, 14, 1, 2, 2, 2, 1, 4, 5, 2, 4, 15, 14, 20, 6, 2, 1, 19, 1, 12, 14, 31, 2, 8, 20, 2, 1, 2, 2, 25, 1, 1, 4, 6, 2, 11, 6, 4, 11, 1, 2, 4, 4, 4, 1, 6, 6, 2, 13, 11, 41, 9, 54, 4, 13, 2, 6, 5, 11, 5, 52, 1, 4, 5, 12, 5, 6, 14, 24, 20, 12, 5, 11, 2, 1, 16, 1, 4, 2, 1, 52, 3, 8, 9, 1, 1, 1, 1, 3, 5, 6, 3, 7, 1, 21, 6, 3, 1, 3, 1, 3, 1, 10, 2, 1, 1, 5, 20, 6, 1, 7, 1, 3, 1, 33, 1, 2, 12, 3, 25, 1, 26, 14, 8, 5, 3, 4, 2, 1, 1, 1, 3, 3, 1, 7, 4, 31, 21, 4, 39, 1, 4, 1, 7, 6, 4, 1, 6, 7, 1, 2, 2, 3, 2, 1, 2, 1, 11, 3, 2, 1, 2, 22, 18, 4, 2, 1, 1, 2, 3, 5, 21, 1, 3, 11, 3, 59, 1, 1, 3, 4, 8, 2, 5, 10, 3, 9, 1, 32, 3, 8, 10, 2, 2, 2, 2, 1, 14, 5, 8, 4, 3, 1, 1, 1, 1, 6, 31, 1, 5, 2, 9, 5, 29, 7, 7, 22, 2, 11, 18, 8, 1, 3, 1, 2, 2, 12, 25, 3, 6, 4, 1, 2, 8, 2, 3, 23, 2, 6, 1, 8, 1, 15, 3, 12, 1, 1, 1, 1, 1, 3, 8, 9, 4, 5, 10, 9, 6, 4, 1, 14, 1, 1, 1, 4, 2, 8, 5, 1, 1, 1, 6, 17, 1, 3, 1, 4, 3, 1, 1, 4, 2, 26, 2, 13, 6, 3, 4, 1, 1, 15, 1, 1, 1, 22, 4, 5, 13, 6, 2, 1, 20, 18, 1, 3, 3, 24, 1, 3, 7, 12, 13, 2, 5, 56], 'id_pos': [73, 15, 115, 2, 1, 5, 1, 1, 9, 46, 5, 19, 4, 129, 9, 2, 10, 1, 32, 4, 1, 1, 25, 44, 29, 7, 131, 12, 43, 129, 1, 40, 60, 27, 5, 6, 22, 18, 1, 43, 9, 2, 134, 11, 5, 10, 4, 3, 41, 9, 47, 1, 65, 87, 176, 1, 45, 3, 165, 35, 69, 7, 17, 1, 6, 64, 5, 25, 47, 2, 34, 15, 1, 1, 2, 2, 3, 6, 3, 1, 1, 41, 30, 2, 2, 1, 5, 13, 10, 6, 3, 33, 27, 10, 2, 3, 3, 2, 7, 3, 16, 3, 3, 3, 2, 6, 2, 11, 1, 4, 4, 13, 32, 2, 15, 1, 32, 23, 25, 5, 9, 1, 4, 1, 2, 1, 4, 14, 2, 1, 8, 4, 3, 2, 5, 17, 6, 1, 2, 19, 33, 1, 1, 1, 1, 1, 1, 2, 6, 2, 1, 10, 8, 1, 7, 2, 2, 2, 3, 2, 3, 1, 2, 11, 5, 6, 6, 21, 32, 2, 10, 3, 2, 1, 2, 1, 2, 9, 1, 3, 1, 1, 1], 'rot_pos': [2, 1, 9, 1, 1, 2, 1, 4, 2, 1, 4, 15, 2, 7, 2, 5, 1, 3, 1, 19, 3, 46, 2, 1, 10, 7, 2, 4, 23, 1, 10, 15, 55, 1, 1, 1, 2, 7, 14, 7, 1, 7, 6, 30, 14, 1, 2, 2, 2, 1, 4, 5, 2, 4, 15, 14, 20, 6, 2, 1, 19, 1, 12, 14, 31, 2, 8, 20, 2, 1, 2, 2, 25, 1, 1, 4, 6, 2, 11, 6, 4, 11, 1, 2, 4, 4, 4, 1, 6, 6, 2, 13, 11, 41, 9, 54, 4, 13, 2, 6, 5, 11, 5, 52, 1, 4, 5, 12, 5, 6, 14, 24, 20, 12, 5, 11, 2, 1, 16, 1, 4, 2, 1, 52, 3, 8, 9, 1, 1, 1, 1, 3, 5, 6, 3, 7, 1, 21, 6, 3, 1, 3, 1, 3, 1, 10, 2, 1, 1, 5, 20, 6, 1, 7, 1, 3, 1, 33, 1, 2, 12, 3, 25, 1, 26, 14, 8, 5, 3, 4, 2, 1, 1, 1, 3, 3, 1, 7, 4, 31, 21, 4, 39, 1, 4, 1, 7, 6, 4, 1, 6, 7, 1, 2, 2, 3, 2, 1, 2, 1, 11, 3, 2, 1, 2, 22, 18, 4, 2, 1, 1, 2, 3, 5, 21, 1, 3, 11, 3, 59, 1, 1, 3, 4, 8, 2, 5, 10, 3, 9, 1, 32, 3, 8, 10, 2, 2, 2, 2, 1, 14, 5, 8, 4, 3, 1, 1, 1, 1, 6, 31, 1, 5, 2, 9, 5, 29, 7, 7, 22, 2, 11, 18, 8, 1, 3, 1, 2, 2, 12, 25, 3, 6, 4, 1, 2, 8, 2, 3, 23, 2, 6, 1, 8, 1, 15, 3, 12, 1, 1, 1, 1, 1, 3, 8, 9, 4, 5, 10, 9, 6, 4, 1, 14, 1, 1, 1, 4, 2, 8, 5, 1, 1, 1, 6, 17, 1, 3, 1, 4, 3, 1, 1, 4, 2, 26, 2, 13, 6, 3, 4, 1, 1, 15, 1, 1, 1, 22, 4, 5, 13, 6, 2, 1, 20, 18, 1, 3, 3, 24, 1, 3, 7, 12, 13, 2, 5, 56], 'id_neg': [1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 6, 2, 5, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 3, 2, 3, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 14, 1, 4, 2, 2, 2, 1, 7, 1, 2, 2, 2, 1, 1, 3, 3, 1, 1, 1, 1, 1, 4, 1, 6, 4, 4, 4, 1, 1, 6, 2, 19, 2, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 8, 6, 6, 2, 1, 1, 1, 8, 6, 1, 1, 1, 7, 1, 3, 5, 10, 2, 1, 1, 1, 3, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 2, 3]}
copter_distr = {'rot_neg': [154, 172, 78, 166, 91, 9, 13, 8, 2, 47, 9, 154, 155, 22, 2, 28, 96, 335, 195, 225, 149, 110, 16, 29, 14, 5, 1, 24, 8, 25, 67, 40, 56, 129, 3, 1, 1, 11, 8, 3, 3, 2, 1, 2, 5, 10, 3, 3, 9, 1, 31, 9, 35, 10, 60, 33, 2, 61, 1, 4, 11, 83, 46, 14, 11, 17, 335, 48], 'rot_pos': [154, 172, 78, 166, 91, 9, 13, 8, 2, 47, 9, 154, 155, 22, 2, 28, 96, 335, 195, 225, 149, 110, 16, 29, 14, 5, 1, 24, 8, 25, 67, 40, 56, 129, 3, 1, 1, 11, 8, 3, 3, 2, 1, 2, 5, 10, 3, 3, 9, 1, 31, 9, 35, 10, 60, 33, 2, 61, 1, 4, 11, 83, 46, 14, 11, 17, 335, 48], 'id_neg': [1, 2, 1, 1, 1, 1, 1, 1, 5, 1, 8, 1, 3, 1, 1, 1, 4, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 2, 8, 2, 3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3, 5, 2, 1, 1, 2, 1, 3, 1, 1, 1, 2, 2, 6, 1, 1, 1, 2, 1, 3, 3, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 2], 'id_pos': [154, 126, 2, 41, 78, 93, 2, 22, 6, 127, 32, 2, 54, 4, 150, 47, 44, 81, 1, 28, 96, 147, 16, 167, 2, 290, 103, 176, 110, 16, 7, 1, 18, 14, 5, 1, 24, 8, 24, 67, 87, 9, 9, 87, 31, 3, 1, 8, 1, 8, 3, 3, 2, 1, 2, 5, 10, 3, 3, 45, 9, 35, 9, 60, 33, 1, 61, 1, 4, 11, 130, 14, 11, 17, 175, 37, 15, 50, 54, 48]}
sphero_distr = {'rot_neg': [5, 13, 5, 2, 2, 4, 1, 2, 1, 1, 1, 1, 3, 5, 5, 1, 1, 2, 3, 1, 3, 2, 2, 4, 1, 2, 1, 5, 5, 3, 3, 1, 1, 3, 2, 1, 3, 1, 2, 2, 1, 1, 1, 1, 4, 2, 3, 4, 2, 1, 1, 1, 5, 1, 3, 1, 5, 2, 6, 2, 1, 1, 1, 1, 1, 4, 2, 1, 2, 5, 14, 1, 2, 1, 1, 1, 4, 15, 1, 2, 1, 5, 1, 1, 8, 1, 1, 2, 1, 1, 4, 4, 8, 1, 4, 3, 1, 1, 2, 1, 1, 4, 2, 1, 15, 4, 1, 1, 2, 1, 2, 5, 2, 1, 2, 2, 1, 3, 2, 1, 11, 1, 2, 4, 6, 6, 1, 1, 3, 1, 14, 26, 4, 1, 1, 1, 2, 3, 2, 1, 5, 2, 1, 1, 1, 3, 1, 1, 1, 1, 1, 4, 2, 2, 4, 2, 4, 2, 2, 2, 2, 2, 2, 1, 1, 1, 4, 11, 1, 1, 1, 1, 1, 3, 1, 3, 11, 1, 1, 1, 2, 1, 1, 1, 5, 1, 1, 2, 2, 2, 1, 2, 2, 2, 3, 2, 2, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 7, 1, 1, 5, 2, 2, 1, 3, 1, 3, 1, 1, 13, 2, 1, 2, 4, 1, 5, 3, 2, 1, 1, 3, 3, 1, 3, 3, 3, 3, 2, 5, 1, 11, 1, 1, 1, 27, 4, 18, 5, 34, 3, 3, 1, 4, 9, 1, 2, 2, 1, 3, 3, 1, 1, 1, 1, 1, 1, 6, 3, 5, 3, 4, 2, 1, 2, 1, 1, 5, 3, 1, 1, 18, 1, 1, 1, 1, 5, 3, 3, 1, 2, 1, 1, 3, 4, 1, 1, 1, 1, 3, 2, 1, 3, 1, 1, 1, 2, 2, 4, 1, 3, 3, 2, 6, 3, 1, 1, 1, 1, 1, 1, 6, 2, 12, 1, 1, 2, 3, 24, 3, 1, 3, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 3, 5, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 3, 3, 4, 3, 1, 1, 1, 1, 5, 1, 4, 2, 1, 2, 6, 5, 3, 1, 1, 2, 3, 1, 2, 1, 1, 1, 3, 2, 2, 2, 3, 12, 1, 5, 4, 3, 2, 6, 2, 4, 6, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 2, 3, 1, 1, 1, 3, 1, 2, 1, 2, 2, 7, 4, 1, 2, 3, 2, 1, 1, 7, 1, 5, 1, 1, 3, 1, 2, 1, 7, 1, 1, 1, 2, 1, 1, 1, 3, 8, 2, 4, 2, 2, 1, 1, 2, 3, 3, 8, 15, 4, 4], 'rot_pos': [5, 13, 5, 2, 2, 4, 1, 2, 1, 1, 1, 1, 3, 5, 5, 1, 1, 2, 3, 1, 3, 2, 2, 4, 1, 2, 1, 5, 5, 3, 3, 1, 1, 3, 2, 1, 3, 1, 2, 2, 1, 1, 1, 1, 4, 2, 3, 4, 2, 1, 1, 1, 5, 1, 3, 1, 5, 2, 6, 2, 1, 1, 1, 1, 1, 4, 2, 1, 2, 5, 14, 1, 2, 1, 1, 1, 4, 15, 1, 2, 1, 5, 1, 1, 8, 1, 1, 2, 1, 1, 4, 4, 8, 1, 4, 3, 1, 1, 2, 1, 1, 4, 2, 1, 15, 4, 1, 1, 2, 1, 2, 5, 2, 1, 2, 2, 1, 3, 2, 1, 11, 1, 2, 4, 6, 6, 1, 1, 3, 1, 14, 26, 4, 1, 1, 1, 2, 3, 2, 1, 5, 2, 1, 1, 1, 3, 1, 1, 1, 1, 1, 4, 2, 2, 4, 2, 4, 2, 2, 2, 2, 2, 2, 1, 1, 1, 4, 11, 1, 1, 1, 1, 1, 3, 1, 3, 11, 1, 1, 1, 2, 1, 1, 1, 5, 1, 1, 2, 2, 2, 1, 2, 2, 2, 3, 2, 2, 1, 3, 3, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 7, 1, 1, 5, 2, 2, 1, 3, 1, 3, 1, 1, 13, 2, 1, 2, 4, 1, 5, 3, 2, 1, 1, 3, 3, 1, 3, 3, 3, 3, 2, 5, 1, 11, 1, 1, 1, 27, 4, 18, 5, 34, 3, 3, 1, 4, 9, 1, 2, 2, 1, 3, 3, 1, 1, 1, 1, 1, 1, 6, 3, 5, 3, 4, 2, 1, 2, 1, 1, 5, 3, 1, 1, 18, 1, 1, 1, 1, 5, 3, 3, 1, 2, 1, 1, 3, 4, 1, 1, 1, 1, 3, 2, 1, 3, 1, 1, 1, 2, 2, 4, 1, 3, 3, 2, 6, 3, 1, 1, 1, 1, 1, 1, 6, 2, 12, 1, 1, 2, 3, 24, 3, 1, 3, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 3, 5, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 3, 3, 4, 3, 1, 1, 1, 1, 5, 1, 4, 2, 1, 2, 6, 5, 3, 1, 1, 2, 3, 1, 2, 1, 1, 1, 3, 2, 2, 2, 3, 12, 1, 5, 4, 3, 2, 6, 2, 4, 6, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 2, 3, 1, 1, 1, 3, 1, 2, 1, 2, 2, 7, 4, 1, 2, 3, 2, 1, 1, 7, 1, 5, 1, 1, 3, 1, 2, 1, 7, 1, 1, 1, 2, 1, 1, 1, 3, 8, 2, 4, 2, 2, 1, 1, 2, 3, 3, 8, 15, 4, 4], 'id_neg': [2, 1, 1, 1, 2, 1, 2, 2, 4, 4, 1, 2, 3, 1, 5, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 5, 1, 2, 3, 1, 4, 1, 1, 1, 2, 7, 1, 1, 1, 1, 1, 1, 1, 3, 4, 1, 1, 5, 1, 1, 5, 1, 1, 1, 3, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 10, 27, 1, 14, 2, 5, 5, 6, 1, 3, 2, 5, 2, 1, 3, 2, 1, 4, 3, 1, 1, 1, 1, 1, 2, 2, 2, 5, 2, 1, 1, 1, 3, 1, 4, 1, 1, 2, 2, 1, 6, 5, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 3, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 2, 1, 1, 1, 1, 3, 2, 1, 2, 2, 1, 7, 1, 4, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, 1, 4, 1, 1, 2, 1, 1, 3, 3, 1, 3, 1, 1, 9, 4, 1, 1, 1, 1, 1, 2, 10, 1, 1, 3, 1, 1, 3, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 3, 1, 1, 2, 4, 3, 1, 1, 1, 1, 1, 2, 2, 1, 5, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 1, 7, 1, 2, 2, 1, 1, 1, 2, 4, 2, 1, 1, 4, 4, 4, 2, 1, 1, 3, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1], 'id_pos': [33, 4, 3, 1, 1, 11, 1, 1, 1, 3, 16, 7, 2, 5, 5, 1, 18, 2, 2, 1, 2, 6, 43, 11, 1, 2, 1, 4, 2, 1, 1, 2, 17, 49, 1, 1, 11, 6, 2, 13, 4, 42, 8, 6, 2, 2, 15, 4, 20, 13, 5, 1, 1, 43, 14, 12, 3, 2, 2, 3, 4, 2, 23, 1, 30, 4, 1, 4, 5, 2, 17, 2, 4, 1, 9, 14, 26, 1, 14, 1, 1, 4, 2, 1, 5, 14, 1, 3, 11, 1, 7, 45, 19, 6, 2, 5, 2, 3, 5, 12, 2, 7, 2, 29, 1, 1, 1, 2, 7, 3, 1, 11, 1, 2, 7, 2, 1, 7, 1, 2, 1, 4, 2, 1, 2, 10, 30, 41, 2, 12, 8, 21, 2, 2, 16, 14, 3, 1, 1, 3, 1, 4, 3, 1, 3, 21, 3, 4, 67, 4, 38, 2, 52, 3, 3, 3, 1, 8, 40, 21, 2, 1, 19, 5, 11, 2, 6, 1, 1, 2, 1, 1, 1, 3, 20, 3, 1, 17, 1, 2, 1, 2, 1, 10, 1, 1, 4, 18, 11, 20, 16, 22, 3, 27, 1, 3, 4, 1, 7, 2, 1, 3, 1, 2, 1, 4, 18, 1, 1, 1, 1, 2, 2, 12, 1, 31, 9, 9, 10, 8, 6, 5, 3, 1, 1, 4, 8, 10, 2, 17, 1, 3, 2, 25, 3, 18, 1, 10, 15, 4, 4, 2, 2, 2, 3, 3, 4, 1, 3, 2, 3, 9, 3, 12, 7, 2, 3, 2, 5, 1, 2, 1, 1, 5, 10, 4, 21, 6, 1, 2, 9, 3, 6, 8, 3, 68]}
youbot_distr = {'rot_neg': [7, 10, 2, 12, 21, 5, 52, 21, 83, 85, 1, 4, 1, 6, 3, 6, 15, 2, 94, 1, 2, 6, 20, 18, 12, 3, 57, 2, 1, 2, 1, 15, 6, 47, 98, 18, 4, 4, 4, 14, 170, 60, 5, 30, 78, 10, 29, 20, 1, 76, 76, 1, 2, 12, 6, 27, 1, 2, 32, 2, 81, 128, 40, 14, 49, 14, 89, 61, 10, 1, 2, 1, 3, 3, 12, 124, 53, 10, 6, 37, 39, 1, 4, 1, 1, 2, 7, 43, 36, 101, 182, 2, 1, 4, 1, 1, 22, 3, 48, 1, 7, 10, 1, 73, 2, 3, 1, 4, 2, 3, 2, 1, 3, 15, 1, 2, 1, 90, 1, 1, 1, 45, 41, 2, 4, 10, 27, 13, 63, 9, 1, 1, 35, 24], 'rot_pos': [7, 10, 2, 12, 21, 5, 52, 21, 83, 85, 1, 4, 1, 6, 3, 6, 15, 2, 94, 1, 2, 6, 20, 18, 12, 3, 57, 2, 1, 2, 1, 15, 6, 47, 98, 18, 4, 4, 4, 14, 170, 60, 5, 30, 78, 10, 29, 20, 1, 76, 76, 1, 2, 12, 6, 27, 1, 2, 32, 2, 81, 128, 40, 14, 49, 14, 89, 61, 10, 1, 2, 1, 3, 3, 12, 124, 53, 10, 6, 37, 39, 1, 4, 1, 1, 2, 7, 43, 36, 101, 182, 2, 1, 4, 1, 1, 22, 3, 48, 1, 7, 10, 1, 73, 2, 3, 1, 4, 2, 3, 2, 1, 3, 15, 1, 2, 1, 90, 1, 1, 1, 45, 41, 2, 4, 10, 27, 13, 63, 9, 1, 1, 35, 24], 'id_neg': [1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 6, 1, 3, 3, 3, 1, 1, 2, 1, 1, 2, 2, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1], 'id_pos': [464, 3, 37, 5, 1, 1, 107, 101, 46, 156, 2, 13, 2, 99, 150, 148, 56, 38, 25, 4, 117, 725, 81, 78, 146, 2, 50, 2, 5, 1, 7, 4, 3, 1, 31, 64, 2, 88, 10, 3, 7, 8, 3, 1, 8, 1, 4, 17, 13, 27, 1, 9, 35, 1, 5, 3, 3, 3, 10, 5, 2, 3, 5, 3, 2, 4, 1, 1, 3, 10, 3, 138, 60]}


copter_voc = {
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/copter_A': 0.9768140230393457,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/copter_B': 0.9817158931082982,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/copter_C': 0.9907514450867052,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/copter_black_black': 0.9651398395683557,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/copter_black_blue': 0.9893617021276596,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/copter_black_green': 0.9780515117581186,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/copter_black_orange': 1.0,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/copter_black_white': 0.9777777777777776,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/copter_blue_blue': 0.9881108600034021,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/copter_blue_white': 0.9753575076608783,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/copter_green_blue': 0.9047619047619048,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/copter_green_green': 0.945054945054945,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/copter_green_white': 0.9934741333934882,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/copter_orange_blue': 0.9893663194444444,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/copter_orange_green': 0.9777777777777776,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/copter_orange_orange': 1.0,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/copter_orange_white': 0.9719804912592307,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/copter_white_white': 1.0,
 'PascalBoxes_Precision/mAP@0.5IOU': 0.9780831184345742
}
sphero_voc = {
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/sphero_black': 0.2570679990073485,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/sphero_bright_blue': 0.85527961717802,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/sphero_bright_green': 0.8413347401908516,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/sphero_bright_red': 0.8367014422536794,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/sphero_bright_white': 0.8683578712756229,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/sphero_dark_blue': 0.41196788624155034,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/sphero_dark_green': 0.660143859144124,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/sphero_dark_red': 0.6311086458207784,
 'PascalBoxes_Precision/mAP@0.5IOU': 0.670245257638997
}
youbot_voc = {
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/youbot_A': 0.933955570997244,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/youbot_B': 0.8420872061877752,
 'PascalBoxes_PerformanceByCategory/AP@0.5IOU/youbot_C': 0.8642098125113838,
 'PascalBoxes_Precision/mAP@0.5IOU': 0.8800841965654677
}
copter_rot_mae = 1.9335732078605037
sphero_rot_mae = 21.263660017346055
youbot_rot_mae = 3.777876631079478

In [ ]:
OUT_DIR = '/home/lukas/Nextcloud/Studium/Bachelorarbeit/Overleaf/Plots'
plt.style.use('default')
from cycler import cycler

d = [copter_distr['id_neg'],sphero_distr['id_neg'],youbot_distr['id_neg']]
w = [np.zeros_like(d[0]) + 1. / len(d[0]),
     np.zeros_like(d[1]) + 1. / len(d[1]),
     np.zeros_like(d[2]) + 1. / len(d[2])]
hatch_cycle = cycler('hatch', ['/', '*', '+', '|'])
n, bins, patches = plt.hist(d, 
         max([max(l) for l in d])-1,
         alpha=0.7,
         align='left',
         label=['copter','youbot','sphero'],
         weights=w,
         fill=False
        )
hatches = ['', '////', '....']
for patch_set, hatch in zip(patches, hatches):
    plt.setp(patch_set, hatch=hatch)
#plt.title('Distribution of Subsequent Wrong ID Classifications')
plt.legend(loc='upper right')
plt.ylabel('Relative Frequency')
plt.xlabel('Number of Subsequent Wrong ID Classifications')
plt.gca().set_xlim([0.5,8.5])
plt.gca().set_aspect(5)
plt.xticks(np.arange(1, 9, 1.0))
plt.savefig('{}/subsequent_wrong_classification_distribution.pdf'.format(OUT_DIR),bbox_inches='tight')
plt.show()